In [21]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [22]:
STATE_NAMES = {
  'AP': 'Andhra Pradesh',
  'AR': 'Arunachal Pradesh',
  'AS': 'Assam',
  'BR': 'Bihar',
  'CT': 'Chhattisgarh',
  'GA': 'Goa',
  'GJ': 'Gujarat',
  'HR': 'Haryana',
  'HP': 'Himachal Pradesh',
  'JH': 'Jharkhand',
  'KA': 'Karnataka',
  'KL': 'Kerala',
  'MP': 'Madhya Pradesh',
  'MH': 'Maharashtra',
  'MN': 'Manipur',
  'ML': 'Meghalaya',
  'MZ': 'Mizoram',
  'NL': 'Nagaland',
  'OR': 'Odisha',
  'PB': 'Punjab',
  'RJ': 'Rajasthan',
  'SK': 'Sikkim',
  'TN': 'Tamil Nadu',
  'TG': 'Telangana',
  'TR': 'Tripura',
  'UT': 'Uttarakhand',
  'UP': 'Uttar Pradesh',
  'WB': 'West Bengal',
  'AN': 'Andaman and Nicobar Islands',
  'CH': 'Chandigarh',
  'DN': 'Dadra and Nagar Haveli and Daman and Diu',
  'DL': 'Delhi',
  'JK': 'Jammu and Kashmir',
  'LA': 'Ladakh',
  'LD': 'Lakshadweep',
  'PY': 'Puducherry',
  'TT': 'India',
 # [UNASSIGNED_STATE_CODE]: 'Unassigned',
}

In [43]:
#Change the date here to get historical dates
date_=datetime.strptime('2021-11-01',"%Y-%m-%d")
dates_list= [date_+timedelta(days=i) for i in range(32)]
dates_list

[datetime.datetime(2021, 11, 1, 0, 0),
 datetime.datetime(2021, 11, 2, 0, 0),
 datetime.datetime(2021, 11, 3, 0, 0),
 datetime.datetime(2021, 11, 4, 0, 0),
 datetime.datetime(2021, 11, 5, 0, 0),
 datetime.datetime(2021, 11, 6, 0, 0),
 datetime.datetime(2021, 11, 7, 0, 0),
 datetime.datetime(2021, 11, 8, 0, 0),
 datetime.datetime(2021, 11, 9, 0, 0),
 datetime.datetime(2021, 11, 10, 0, 0),
 datetime.datetime(2021, 11, 11, 0, 0),
 datetime.datetime(2021, 11, 12, 0, 0),
 datetime.datetime(2021, 11, 13, 0, 0),
 datetime.datetime(2021, 11, 14, 0, 0),
 datetime.datetime(2021, 11, 15, 0, 0),
 datetime.datetime(2021, 11, 16, 0, 0),
 datetime.datetime(2021, 11, 17, 0, 0),
 datetime.datetime(2021, 11, 18, 0, 0),
 datetime.datetime(2021, 11, 19, 0, 0),
 datetime.datetime(2021, 11, 20, 0, 0),
 datetime.datetime(2021, 11, 21, 0, 0),
 datetime.datetime(2021, 11, 22, 0, 0),
 datetime.datetime(2021, 11, 23, 0, 0),
 datetime.datetime(2021, 11, 24, 0, 0),
 datetime.datetime(2021, 11, 25, 0, 0),
 datetime

In [44]:
source_path="../RAWCSV"

In [45]:
for Date in dates_list:
    tdate = Date
    date=tdate.strftime("%d/%m/%Y")
    if not isinstance(Date,str):
        Date=Date.strftime("%Y-%m-%d")
    print(Date)
    # /home/swiadmin/test/csv/latest/
    cowin_vaccine_data_districtwise_prev=pd.read_csv("/home/swiadmin/test/csv/latest/cowin_vaccine_data_districtwise.csv",header=[0,1])
    try:
        cowin_vaccine_data_districtwise_prev = cowin_vaccine_data_districtwise_prev.drop([date],axis=1)
    except:
        pass
    cowin_vaccine_data_districtwise_master=pd.DataFrame()
    for state in STATE_NAMES.keys():
        if state=="TT":
            continue
#         print(state)
        state_=pd.read_csv(f"{source_path}/{Date}/{state}_final.csv")
        state_.dropna(subset=["District"],axis=0,inplace=True)
        state_.reset_index(inplace=True,drop=True)
        temp_state_df=pd.DataFrame()
        temp_state_df["District"]=state_["District"]
        temp_state_df["State_Code"]=state
        temp_state_df["State"]=temp_state_df["State_Code"].map(STATE_NAMES)
        temp_state_df["District_Key"]=temp_state_df.apply(lambda rw: rw["State_Code"]+"_"+rw["District"],axis=1)
        temp_state_df["First Dose Administered"]=state_["cumulativeVaccinated1NumberForDistrict"]
        temp_state_df["Second Dose Administered"]=state_["cumulativeVaccinated2NumberForDistrict"]
        temp_state_df["Total Dose Administered"]=temp_state_df["First Dose Administered"]+temp_state_df["Second Dose Administered"]
        cowin_vaccine_data_districtwise_master=pd.concat([cowin_vaccine_data_districtwise_master,temp_state_df],axis=0)
    
    # if not isinstance(Date,str):
    

    columns=pd.MultiIndex.from_tuples(zip(["District","State_Code","State",
                "District_Key",date,date,date],
               [" "," "," "," ","Total Doses Administered",
                "First Dose Administered","Second Dose Administered"
               ]))
    cowin_vaccine_data_districtwise_master.columns=columns
    cowin_vaccine_data_districtwise_master.reset_index(inplace=True,drop=True)
    
    cowin_vaccine_data_districtwise_latest=pd.merge(cowin_vaccine_data_districtwise_prev,cowin_vaccine_data_districtwise_master,
                                                   left_on=[("State_Code"," "),("State"," "),("District_Key"," "),("District"," ")],
                                                   right_on=[("State_Code"," "),("State"," "),("District_Key"," "),("District"," ")],
                                                   how="left")
    cowin_vaccine_data_districtwise_latest = cowin_vaccine_data_districtwise_latest.drop_duplicates()
    cowin_vaccine_data_districtwise_latest.to_csv("/home/swiadmin/test/csv/latest/cowin_vaccine_data_districtwise.csv",index=False)

2021-11-01
2021-11-02
2021-11-03
2021-11-04
2021-11-05
2021-11-06
2021-11-07
2021-11-08
2021-11-09
2021-11-10
2021-11-11
2021-11-12
2021-11-13
2021-11-14
2021-11-15
2021-11-16
2021-11-17
2021-11-18
2021-11-19
2021-11-20
2021-11-21
2021-11-22
2021-11-23
2021-11-24
2021-11-25
2021-11-26
2021-11-27
2021-11-28
2021-11-29
2021-11-30
2021-12-01
2021-12-02
